# Doge

训练 [Wonderful Matrices](https://arxiv.org/abs/2412.11834) 论文中提出的 `Doge` 小型语言模型.
Doge在 Transformers 的框架基础上, 将序列变换部分的 `Multi-Head Attention` 替换为 `Dynamic Mask Attention`, 将状态变换部分的 `MLP` 替换为 `CDMoE` . 

![doge_architecture](../assets/doge_architecture.png)

## 预训练

### 下载预训练数据集


预训练数据集, 我们选取了 `fineweb-edu-dedup` 高质量文本, `cosmopedia-v2` 合成指令数据集, 并补充 `python-edu` 与 `fine-math` 来保证模型的代码与数学能力. 


> 请注意: 由于数据集过大, 至少需要 2TB 的存储空间. 如果您的存储空间不足, 可以自行在[这里](./utils/download_pt_datasets.py)选择部分数据集进行下载. 

In [ ]:
# 填写保存路径, 缓存路径和进程数
!python ./examples/utils/download_pt_datasets.py --save_dir ./datasets --cache_dir ./cache --num_proc 1

### 预处理数据集


我们需要使用 `tokenizer` 将数据集转为模型可接受的 `input_ids` 与 `attention_mask`.
如果使用 `LlamaTokenizer` , 该 tokenizer 词表大小为 `32768` , 使用 `[INST]` 与 `[/INST]` 标记指令. 它还包括工具标记, 但是我们不会在这里使用它们.
像 cosmopedia-v2 这样的数据集就包括 `prompt` 与 `text` 两个字段, 我们就将他们标记为用户内容与助手内容.


```python
conversation = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": text},
]
return tokenizer.apply_chat_template(conversation, tokenize=True, padding='max_length', truncation=True, max_length=MAX_LENGTH, return_dict=True)
```


当然你也可以自行加入一些指令提示.


```python
conversation = [
    {"role": "user", "content": "Who are you?"},
    {"role": "assistant", "content": "I am an AI assistant named `Doge`, I am a language model trained by `Shi Jingze` based on the `Doge` architecture, and my task is to provide appropriate answers and support to users based on their questions and requests."},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": text},
]
```

在这里我们推荐使用 [Doge-tokenizer](https://huggingface.co/SmallDoge/Doge-tokenizer) 来处理数据集, 它是由 `Llama-3.3` 的分词器针对 `smollm-corpus` 训练得到的, 词表大小为 `32768` , 训练脚本可以在 [这里](./examples/utils/train_tokenizer_from_old.py) 找到.

In [ ]:
# 填写数据集路径, 保存路径, 分词器路径, 样本数量, 最大长度和进程数
# NOTE: 我们只保留 256B tokens 的数据集, 比例为 fineweb-edu:cosmopedia-v2:python-edu:open-web-math = 7:2:0.5:0.5, 如果你需要训练更大的模型, 请自行增加数据集的规模
!python ./examples/utils/preprocess_pt_datasets.py --datasets_dir ./datasets --save_dir ./datasets --tokenizer_name_or_path SamllDoge/Doge-tokenizer --train_examples 128000000 --test_examples 1000 --max_length 2048 --num_proc 16

### 合并数据集


我们将 fineweb-edu_tokenized, cosmopedia-v2, python-edu 和 finemath 数据集合并为 `pretrain` 数据集.
然后将它们打乱顺序 `seed=233` , 并拆分出来 `1,000` 个样本作为测试集.

In [ ]:
# 填写数据集路径, 保存路径, 样本数量和进程数
!python ./examples/utils/concatenate_pt_datasets.py --datasets_dir ./datasets --save_dir ./datasets --train_examples 128000000 --test_examples 1000 --num_proc 16

### 配置模型参数


我们配置一个 `20M` 的小型模型, 进行训练测试.


| Model | Params | n_layers | d_model | d_ff | n_heads | kv_heads | n_exprets | n_expert_heads | n_expert_pre_head |
|---|---|---|---|---|---|---|---|---|---|
| Doge-20M | 13M | 8 | 256 | 512 | 2 | 1 | - | - | - |
| Doge-60M | 54M | 16 | 512 | 1024 | 4 | 2 | - | - | - |
| Doge-160M | 152M | 24 | 768 | 1536 | 6 | 3 | - | - | - |
| Doge-320M | 335M | 32 | 1024 | 2048 | 8 | 4 | - | - | - |

- n_layers 是模型的解码器层数
- d_model 是模型的隐藏层维度
- n_heads 是多头注意力头数 d_model // n_heads 最好保持在 64 以上


> `Doge-MoE` 模型可以继承 `Doge` 模型的密集激活参数, 并通过设置 `n_experts`, `n_expert_heads`, `n_expert_pre_head` 来增加稀疏激活的参数.

### 配置预训练超参数

| Model | tokens | max_train_steps | accumulate_steps | learning_rate | scheduler | warmup_ratio | decay_ratio | weight_decay | min_lr_rate |
|---|---|---|---|---|---|---|---|---|---|
| Doge-20M | 4B | 8,000 | 256 | 8e-3 | warmup_stable_decay | 0.1 | 0.1 | 0.01 | 0.0 |
| Doge-60M | 16B | 16,000 | 512 | 6e-3 | warmup_stable_decay | 0.1 | 0.1 | 0.01 | 0.0 |
| Doge-160M | 32B | 24,000 | 768 | 4e-3 | warmup_stable_decay | 0.1 | 0.1 | 0.01 | 0.0 |
| Doge-320M | 64B | 32,000 | 1024 | 2e-3 | warmup_stable_decay | 0.1 | 0.1 | 0.01 | 0.0 |

> 根据 [SmolLM博客](https://huggingface.co/blog/smollm) 的经验, 我们将 [Chinchilla](https://arxiv.org/pdf/2203.15556) 中参数与标记的缩放比例扩大 10 倍.

> 使用 `warmup_stable_decay` 是为了随时使用检查点在更大的数据集上继续训练, 参见 [Scaling Laws and Compute-Optimal Training Beyond Fixed Training Durations](https://arxiv.org/pdf/2405.18392).

### 预训练模型

In [ ]:
# 你需要指定配置文件路径, 所有参数都在配方配置文件中
!ACCELERATE_LOG_LEVEL=info accelerate launch ./src/small_doge/pt.py --config_file recipes/accelerate_configs/single_gpu.yaml --config recipes/doge/Doge-20M/config_full.yaml

### 使用


在完成训练后, 我们可以使用 `Transformers` 的 `AutoModelForCausalLM` 加载模型, 并使用 `AutoTokenizer` 加载 `LlamaTokenizer` .

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("SmallDoge/Doge-20M")
model = AutoModelForCausalLM.from_pretrained("SmallDoge/Doge-20M", trust_remote_code=True)

In [ ]:
inputs = tokenizer("Hey how are you doing?", return_tensors="pt")

out = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.batch_decode(out))

## 指令微调

### 下载微调数据集


微调数据集, 我们选取了 `smoltalk` 数据集, 来进行监督微调, `ultrafeedback_binarized` 数据集, 来进行偏好优化.

In [ ]:
# 填写保存路径, 缓存路径和进程数
!python ./examples/utils/download_ft_datasets.py --save_dir ./datasets --cache_dir ./cache --num_proc 1

### 处理微调数据集


我们将微调数据集应用于`聊天模板`.

In [ ]:
# 填充数据集存放路径 数据集保存路径、分词器路径、进程数量 
!python ./examples/utils/preprocess_ft_datasets.py --datasets_dir ./datasets --save_dir ./datasets --tokenizer_name_or_path SmallDoge/Doge-tokenizer --num_proc 8

### 合并数据集

如果你下载了更多数据集进行微调, 我们需要将其合并打乱来混合在一起.

In [ ]:
# 填写数据集路径, 保存路径, 样本数量和进程数
!python ./examples/utils/concatenate_ft_datasets.py --datasets_dir ./datasets --save_dir ./datasets --num_proc 16

### 监督微调模型

我们首先对模型进行监督微调, 使其更够跟随 `prompt` 来生成回复.

In [ ]:
# 你需要指定配置文件路径, 所有参数都在配方配置文件中
!ACCELERATE_LOG_LEVEL=info accelerate launch ./src/small_doge/sft.py --config_file recipes/accelerate_configs/single_gpu.yaml --config recipes/doge/Doge-20M-Instruct/sft/config_full.yaml

### 偏好优化模型

然后我们对监督微调后的模型进行强化学习, 来与人类偏好对齐, 这里使用的是 `DPO` 算法.

In [ ]:
# 你需要指定配置文件路径, 所有参数都在配方配置文件中
!ACCELERATE_LOG_LEVEL=info accelerate launch ./src/small_doge/dpo.py --config_file recipes/accelerate_configs/single_gpu.yaml --config recipes/doge/Doge-20M-Instruct/dpo/config_full.yaml

### 使用

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextStreamer

tokenizer = AutoTokenizer.from_pretrained("SmallDoge/Doge-20M-Instruct")
model = AutoModelForCausalLM.from_pretrained("SmallDoge/Doge-20M-Instruct", trust_remote_code=True)

In [12]:
generation_config = GenerationConfig(
      max_new_tokens=100, 
      use_cache=True, 
      do_sample=True, 
      temperature=0.8, 
      repetition_penalty=1.0
)
steamer = TextStreamer(
      tokenizer=tokenizer, 
      skip_prompt=True
)

In [ ]:
prompt = "Hi, how are you doing today?"

conversation = [
      {"role": "user", "content": prompt}
]
inputs = tokenizer.apply_chat_template(
    conversation=conversation,
    tokenize=True,
    return_tensors="pt",
)

outputs = model.generate(
    inputs, 
    tokenizer=tokenizer,
    generation_config=generation_config, 
    streamer=steamer
)

## 评估

参见 [evaluation](../evaluation/README.md).